# File Hash Enrichment

Playbook to query muliple malware repositories for some basic analysis on a given file hash. 

Notable Tools Used:
* [Malwoverview](https://github.com/alexandreborges/malwoverview)
* [Vt-py](https://github.com/VirusTotal/vt-py)
* [Intezer](https://analyze.intezer.com/)
* [Valhalla](https://valhalla.nextron-systems.com/)
* [Zscaler Threat Library](https://https://threatlibrary.zscaler.com/)

## Setup

Prerequisite - a **.env** file must exist in the same folder as the notebook. The file should have the **VT_API** variable set with a VirusTotal API key.

Loads required packages and variables.

In [1]:
%%capture

# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install vt-py jmespath python-dotenv

import vt
import nest_asyncio
import json
import jmespath
import requests
from dotenv import load_dotenv
from termcolor import colored
import os

nest_asyncio.apply()
load_dotenv()

VT_API = os.getenv('VT_API')

## Input File Hash

In [7]:
IOC =  "274beb57ae19cbc5c2027e08cb2b718dea7ed1acb21bd329d5aba33231fb699d"

## Malwoverview: Get summary from VirusTotal, Hybrid Analysis, OTX and Bazaar

Uses Malwoverview to query given file hash against various malware repositories.

In [ ]:
# VirusTotal and Hybrid Analysis

!malwoverview -H {IOC}

In [ ]:
# OTX
!malwoverview -n 4 -N {IOC}

In [ ]:
# Bazaar
!malwoverview -b 1 -B {IOC}

## VirusTotal: Get File Data

Queries VirusTotal to retrieve data on given file hash.

In [14]:
vt_client = vt.Client(VT_API)
vt_file = vt_client.get_object("/files/" + IOC)
vt_sigma = vt_client.get_object("/files/" + IOC + "/sigma_analysis")
vt_client.close()

## VirusTotal: List File Details

List some details from VT.

In [3]:
print("\nExiftool Properties\n=============================")
for attribute, value in vt_file.exiftool.items(): 
    print(attribute + " : " + value)

print("\nVirusTotal Tags\n=============================")
print(vt_file.tags)


print("\nOther Details\n=============================")
print("First Submission Date: ", vt_file.first_submission_date)
print("Creation Date: ", vt_file.creation_date)
print("Last Modification Date: ", vt_file.last_modification_date)
print("Popular Threat Classification: ", vt_file.popular_threat_classification['suggested_threat_label'])

print("\nNames\n=============================")
for name in vt_file.names:
    print(name)



Exiftool Properties
SubsystemVersion : 6.0
InitializedDataSize : 203264
ImageVersion : 0.0
FileSubtype : 0
FileVersionNumber : 1.0.0.1
LanguageCode : English (U.S.)
InternalName : Service.exe
FileDescription : Host Process for Windows Services
ImageFileCharacteristics : Executable, 32-bit
CharacterSet : Unicode
LinkerVersion : 14.27
FileTypeExtension : exe
OriginalFileName : Service.exe
MIMEType : application/octet-stream
LegalCopyright : Microsoft.  All rights reserved.
FileVersion : 1.0.0.1
TimeStamp : 2020:11:30 10:51:37+00:00
FileType : Win32 EXE
PEType : PE32
FileFlagsMask : 0x003f
ProductVersion : 1.0.0.1
UninitializedDataSize : 0
OSVersion : 6.0
FileOS : Windows NT 32-bit
Subsystem : Windows GUI
MachineType : Intel 386 or later, and compatibles
CompanyName : Microsoft
CodeSize : 473600
ProductName : Host Service
ProductVersionNumber : 1.0.0.1
EntryPoint : 0x30952
ObjectFileType : Executable application

VirusTotal Tags
['peexe', 'invalid-rich-pe-linker-version', 'runtime-module

## VirusTotal: List Triggered Detection Rules

Parses the returned VT file object for Yara and IDS rules.

In [19]:
print("Yara Rules:")
print(jmespath.search('crowdsourced_yara_results[*].rule_name', vt_file))
print()
print("IDS Rules:")
print(jmespath.search('crowdsourced_ids_results[*].rule_msg', vt_file))
print()
print("Sigma Rules:")
print(jmespath.search('rule_matches[*].rule_title', vt_sigma))

# print(jmespath.search('sigma_analysis_summary', vt_file))


Yara Rules:
None

IDS Rules:
['ET INFO TLS Handshake Failure', 'SSLBL: Malicious JA3 SSL-Client Fingerprint detected (Tofsee)']

Sigma Rules:
['Monitoring For Persistence Via BITS', 'Always Install Elevated Windows Installer', 'Whoami Execution', 'Monitoring For Persistence Via BITS', 'Bitsadmin Download', 'Local Accounts Discovery', 'Hiding Files with Attrib.exe', 'Non Interactive PowerShell']


## Malwoverview: Lookup SHA256 on Valhalla

Uses Malwoverview to lookup SHA256 returned by VT against the Valhalla Rule Feed.

In [20]:
!malwoverview -E 4 -C {IOC}


Rulename:   MAL_RAT_Packer_Svehost
Positives:  15
Timestamp:  Mon, 01 Mar 2021 16:14:42 GMT
Total:      70
Tags:       MAL T1045 FILE EXE 

Rulename:   MAL_RAT_Svehost_Mar21_1
Positives:  32
Timestamp:  Wed, 03 Mar 2021 00:19:06 GMT
Total:      71
Tags:       HKTL FILE MAL EXE T1197 

Rulename:   MAL_MarkiRAT_Mar21_1
Positives:  42
Timestamp:  Sat, 08 May 2021 11:07:20 GMT
Total:      70
Tags:       MAL T1197 EXE 



## Malwoverview: Lookup Rule on Valhalla

* Uses Malwoverview to lookup a given rule name against the Valhalla Rule Feed. The code is printed if the rule exists.
* **Manual input: rule_name**

In [21]:
rule_name = "MarkiRAT"
!"malwoverview" -E 1 -C {rule_name}


Yara rules for the provided argument haven't been found on Valhalla!



## Intezer: Lookup SHA256 Analysis Report

Searches for the SHA256 hash returned by VT against Intezer's API.

In [22]:
file_sha256 = vt_file.sha256
headers = {
    'Content-Type': 'application/json'
}
intezer_request = requests.get("https://analyze.intezer.com/api/v1-2/code-items/" + file_sha256 + "/latest-public-analysis", headers=headers)
intezer_response = intezer_request.json()
analysis_id = intezer_response['analysis_id']
root_analysis_id = intezer_response['root_analysis']['analysis_id']
intezer_analysis_strings_req = requests.get("https://analyze.intezer.com/api/v1-2/analyses/" + analysis_id + "/sub/" + root_analysis_id + "/families-by-strings", headers=headers)
intezer_analysis_response = intezer_analysis_strings_req.json()

## Intezer: List Interesting Strings

Parses the returned Intezer analysis object for any interesting strings.

In [23]:
for string_item in intezer_analysis_response['result']['strings']:
    if string_item['software_type'] != "common":
        fam_name = jmespath.search('families[*].family_name', string_item)
        print(str(fam_name) + " : " + string_item['string_value'])

['Generic Malware'] : D:\mklgs\mfcmklg\Release\mfcmklg.pdb
['Generic Malware'] : bitsadmin /addfile pdj "http://microsoft.com-view.space/i.php?u=
['Generic Malware'] : C:\Program Files (x86)\Telegram Desktop
['Generic Malware'] : taskkill /im svehost.exe /t /f
['Generic Malware'] : taskkill /im keepass.exe /t /f
['Generic Malware'] : \AppData\Windows\svehost.exe
['Generic Malware'] : <br>Target directory is
['Generic Malware'] : bitsadmin /create pdj
['Generic Malware'] : error: path is long
['Generic Malware'] : /up/uploadx.php?u=
['Generic Malware'] : uploadsf
['Generic Malware'] : cberror3
['Generic Malware'] : cberror1
['Generic Malware'] : error&r=
['Generic Malware'] : cberror2
None : 0H0O0x0s1
None : 6f7+8X8
['Generic Malware'] : Content-Type: multipart/form-data; boundary=HBA799EB-D9A2-472B-AE86-568D4645707E
['Generic Malware'] : Ad:\agent\_work\3\s\src\vctools\vc7libs\ship\atlmfc\include\afxwin1.inl
['Generic Malware'] : Global\{2194ABA1-BFFA-4e6b-8C26-D1BB20190312}
['Generic 

## Zscaler: Lookup Proxy Detection Signatures

* Search for signatures by malware family or adversary within the Zscaler Threat Library
* **Manual input(s):** Threat Name, Signature ID

In [6]:
zscaler_api_endpoint = "https://api.threatlibrary.zscaler.com/threats"
sig = "markirat"
headers = {'content-type': 'application/json'}
zscaler_req_threatname = requests.get(zscaler_api_endpoint, params={'threatname': sig}, headers=headers)
print('Results for Threat Name:\n', jmespath.search('results[*].threatName', zscaler_req_threatname.json()))
# print()
# zscaler_req_sigid = requests.get(zscaler_api_endpoint, params={'sigid': sig}, headers=headers)
# print("Results for Signature ID:\n", jmespath.search('results[*].threatName', zscaler_req_sigid.json()))

Results for Threat Name:
 ['Win32.Backdoor.Markirat.LZ', 'Win32.Backdoor.MarkiRAT']
